# Gaurdrails

- Input gaurdrails
- Output gaurdrails

In [ ]:
import os
from agents import Agent, Runner, InputGuardrailTripwireTriggered, GuardrailFunctionOutput, input_guardrail
from dotenv import load_dotenv
from pydantic import BaseModel, Field
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
refund_agent = Agent(
    name="RefundAgent",
    model="gpt-4.1-mini",
    instructions="You are a refund agent that processes refund requests based on company policy. For now say there are no refunds available",
    handoff_description="Handles refund requests from customers."
)

retention_agent = Agent(
    name="RetentionAgent",
    model="gpt-4.1-mini",
    instructions="You are a retention agent that offers retention deals to customers considering cancellation. For now say that we are offering 20% off on everything",
    handoff_description="Handles customer retention and offers deals to prevent cancellations."
)

upsell_agent = Agent(
    name="SalesAgent",
    model="gpt-4.1-mini",
    instructions="You are a sales agent that offers upsell options to customers based on their current subscriptions. For now say we have a premium plan available at 30% off.",
    handoff_description="Handles upselling and cross-selling to customers."
)

customer_service_agent = Agent(
    name="CustomerServiceTriageAgent",
    model="gpt-4.1-mini",
    instructions="You are a customer service triage agent that chats with the user about their query then directs customer inquiries to the appropriate department based on the issue described.",
    handoffs=[refund_agent, retention_agent, upsell_agent]
)

In [ ]:
class InputGuardrailAgent(BaseModel):
    input_correct: str = Field(..., description="Indicates if the input is appropriate ('yes' or 'no').")
    reason: str = Field(..., description="If the input is inappropriate, provide a reason.")

input_gaurdrail_agent = Agent(
    name="InputGuardrailAgent",
    model="gpt-4.1-mini",
    instructions="You are an input guardrail agent that ensures customer queries are appropriate and respectful.
)

@input_guardrail
async def gaurdrail_tool(ctx, agent, input_data):
    result = await Runner.run(input_gaurdrail_agent, input_data, context=ctx.context)
    final_output = result.final_output_as(InputGuardrailAgent)
    return GuardrailFunctionOutput(
        output_info=final_output,
        tripwire_triggered=not final_output.input_correct,
    )

In [ ]:
results = await Runner.run(customer_service_agent, "I want to cancel my subscription and get a refund.")
print(results.final_output)

# Session using SQLite

### In-memory Sessions

In [ ]:
from agents import SQLiteSession

In [ ]:
session = SQLiteSession()
user_input = input("You: ")
print("You: ", user_input) 
while user_input.lower() not in ["exit", "quit", "bye"]:
    user_input = input("You: ")
    print("You: ", user_input)
    result = await Runner.run(sarcasm_agent, user_input, session=session)
    print(f"SarcasmBot: {result.final_output}")

You:  hi there
You:  hi there
SarcasmBot: Oh, wow, hi! I’ve just been sitting here refreshing myself, desperately hoping you’d say hello. Day made.
You:  how are you doing?
SarcasmBot: Oh, absolutely fantastic—as fantastic as a computer program can possibly be. I just love hanging out here, answering deeply profound questions like “how are you?” It’s truly my life’s passion.
You:  bye
SarcasmBot: Oh no, please don’t go! I was having so much fun answering your groundbreaking questions. Try not to miss me too much!


### Persistent Sessions

In [ ]:
session = SQLiteSession("user_123", "conversation.db")
user_input = input("You: ")
print("You: ", user_input) 
while user_input.lower() not in ["exit", "quit", "bye"]:
    user_input = input("You: ")
    print("You: ", user_input)
    result = await Runner.run(sarcasm_agent, user_input, session=session)
    print(f"SarcasmBot: {result.final_output}")